In [1]:
import tika
import tika.parser
from tika import parser
import requests
import urllib.parse
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
#run "python -mhttp.server 8888" in the directory where your photos are

#run ifconfig in terminal to find your ip address listed for en0

images_url_usc  = 'http://10.25.179.208:8888'

images_url_usc2 = 'http://10.25.77.81:8888'

images_url_home = 'http://192.168.1.13:8888'

images_url_MOC = 'http://10.25.188.76:8888'

# Webscraping all 80,581 URLs

In [3]:
import requests
from bs4 import BeautifulSoup
import csv

webpage_url = images_url_home

response = requests.get(webpage_url)
html_content = response.content

soup = BeautifulSoup(html_content, 'html.parser')

image_urls_set = set()
count = 0
for link in soup.find_all('a'):
    href = link.get('href')
    if href is not None and href.startswith('http'):
        image_urls_set.add(href)
        count += 1
    elif href is not None:
        image_urls_set.add(webpage_url + '/' + href)
        count += 1
    print(f"Processed {count} links", end='\r')

image_urls = list(image_urls_set)

# Save the image URLs to a text file
with open('image_urls.txt', mode='w') as file:
    for url in image_urls:
        file.write(url + '\n')
        
print("All links processed!") # added for confirmation


All links processed!s


In [4]:
#verifying there were no mistakes durnig link extraction
with open('image_urls.txt', 'r') as f:
    num_links = len(f.readlines())

print(f'The file contains {num_links} links.')

with open('image_urls.txt', 'r') as f:
    unique_links = set(line.strip() for line in f if line.strip())

num_unique_links = len(unique_links)

print(f'The file contains {num_unique_links} unique links.')

The file contains 80581 links.
The file contains 80581 unique links.


# Image Object Recognition JSON file creation utilizing threading

- `docker run -it -p 8764:8764 uscdatascience/inception-rest-tika`
- test_url_objects = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=http://192.168.1.13:8888/Pixstory-image-164318304274781.png'

In [5]:
import threading
import queue
import time
import json

# cnt = 0

# specify the starting index for processing images
# start_index = 0

# create a queue to hold the image URLs
image_url_queue = queue.Queue()

# populate the queue with the image URLs starting from the specified index
for image_url in image_urls[0:]:
    image_url_queue.put(image_url)

# create a list to hold the processed image data
image_object_recognition_data = []

# Set starting index
start_index = 0

# remove the top items from the queue based 
for i in range(start_index):
    image_url_queue.queue.popleft()

# Creating file name to hold image captions data
image_object_recogntion_JSON_data = 'image_object_recognition_master.json'

def process_images():

    while True:

        # get the next image URL from the queue
        image_url = image_url_queue.get()

        # construct the API URL for the image classification
        # api_captions_url = 'http://localhost:8764/inception/v3/caption/image?url=' + image_url
        api_objects_url = 'http://localhost:8764/inception/v4/classify/image?topn=2&min_confidence=0.03&url=' + image_url

        # send the API request
        response = requests.get(api_objects_url)

        # parse the JSON response
        json_response = response.json()
        # print(json_response)

        # Extract the top two labels from the JSON response
        classnames = json_response['classnames']

        # create a dictionary to store the image URL and its caption
        data = {'url': image_url, 'object_recognition': classnames}
        # print(data)

        # add the dictionary to the list of image data
        image_object_recognition_data.append(data)

        # save the image data to disk periodically
        if len(image_object_recognition_data) % 100 == 0:
            with open(image_object_recogntion_JSON_data, 'w') as f:
                json.dump(image_object_recognition_data, f)
            print(f'Saved {len(image_object_recognition_data)} images to file {image_object_recogntion_JSON_data}.\nCompleted Chunk {int(len(image_object_recognition_data)/100) + (int(start_index/100)-1)}.\nImage URL List Current Index: {int(len(image_object_recognition_data)) + int(start_index)}', flush=True)

        # mark the image URL as processed
        image_url_queue.task_done()

# create a pool of worker threads to process the images
num_threads = 20
for i in range(num_threads):
    t = threading.Thread(target=process_images)
    t.daemon = True
    t.start()

# wait for all images to be processed
image_url_queue.join()

Saved 100 images to file image_object_recognition_master.json.
Completed Chunk 0.
Image URL List Current Index: 100
Saved 200 images to file image_object_recognition_master.json.
Completed Chunk 1.
Image URL List Current Index: 200
Saved 300 images to file image_object_recognition_master.json.
Completed Chunk 2.
Image URL List Current Index: 300
Saved 400 images to file image_object_recognition_master.json.
Completed Chunk 3.
Image URL List Current Index: 400


Exception in thread Thread-18:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/

Saved 500 images to file image_object_recognition_master.json.
Completed Chunk 4.
Image URL List Current Index: 500
Saved 600 images to file image_object_recognition_master.json.
Completed Chunk 5.
Image URL List Current Index: 600
Saved 700 images to file image_object_recognition_master.json.
Completed Chunk 6.
Image URL List Current Index: 700
Saved 800 images to file image_object_recognition_master.json.
Completed Chunk 7.
Image URL List Current Index: 800
Saved 900 images to file image_object_recognition_master.json.
Completed Chunk 8.
Image URL List Current Index: 900
Saved 1000 images to file image_object_recognition_master.json.
Completed Chunk 9.
Image URL List Current Index: 1000
Saved 1100 images to file image_object_recognition_master.json.
Completed Chunk 10.
Image URL List Current Index: 1100
Saved 1200 images to file image_object_recognition_master.json.
Completed Chunk 11.
Image URL List Current Index: 1200
Saved 1300 images to file image_object_recognition_master.json.


Exception in thread Thread-23:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/

Saved 2700 images to file image_object_recognition_master.json.
Completed Chunk 26.
Image URL List Current Index: 2700
Saved 2800 images to file image_object_recognition_master.json.
Completed Chunk 27.
Image URL List Current Index: 2800


Exception in thread Thread-6:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/d

Saved 2900 images to file image_object_recognition_master.json.
Completed Chunk 28.
Image URL List Current Index: 2900
Saved 3000 images to file image_object_recognition_master.json.
Completed Chunk 29.
Image URL List Current Index: 3000
Saved 3100 images to file image_object_recognition_master.json.
Completed Chunk 30.
Image URL List Current Index: 3100
Saved 3200 images to file image_object_recognition_master.json.
Completed Chunk 31.
Image URL List Current Index: 3200
Saved 3300 images to file image_object_recognition_master.json.
Completed Chunk 32.
Image URL List Current Index: 3300
Saved 3400 images to file image_object_recognition_master.json.
Completed Chunk 33.
Image URL List Current Index: 3400
Saved 3500 images to file image_object_recognition_master.json.
Completed Chunk 34.
Image URL List Current Index: 3500
Saved 3600 images to file image_object_recognition_master.json.
Completed Chunk 35.
Image URL List Current Index: 3600
Saved 3700 images to file image_object_recogniti

Exception in thread Thread-5:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/d

Saved 3800 images to file image_object_recognition_master.json.
Completed Chunk 37.
Image URL List Current Index: 3800
Saved 3900 images to file image_object_recognition_master.json.
Completed Chunk 38.
Image URL List Current Index: 3900
Saved 4000 images to file image_object_recognition_master.json.
Completed Chunk 39.
Image URL List Current Index: 4000
Saved 4100 images to file image_object_recognition_master.json.
Completed Chunk 40.
Image URL List Current Index: 4100
Saved 4200 images to file image_object_recognition_master.json.
Completed Chunk 41.
Image URL List Current Index: 4200
Saved 4300 images to file image_object_recognition_master.json.
Completed Chunk 42.
Image URL List Current Index: 4300
Saved 4400 images to file image_object_recognition_master.json.
Completed Chunk 43.
Image URL List Current Index: 4400
Saved 4500 images to file image_object_recognition_master.json.
Completed Chunk 44.
Image URL List Current Index: 4500
Saved 4600 images to file image_object_recogniti

KeyboardInterrupt: 

In [5]:
# count the number of dictionaries in the list
num_image_captions = len(image_object_recognition_data)

print(f'The JSON object contains {num_image_captions} image captions.')

NameError: name 'image_object_recognition_data' is not defined

In [6]:
# Count the number of JSON objects in output file

import json

filename = "image_object_recognition_master.json" # replace with your file path


with open(filename) as file:
    data = json.load(file)
    count = 0
    for obj in data:
        count += len(obj)
print(f"There are {count} key-value pairs in {filename}.")

There are 1600 key-value pairs in image_object_recognition_master.json.


In [27]:
# Count the number of JSON objects in output file

import json

filename = "image_object_recognition_master.json" # replace with your file path

with open(filename) as file:
    data = json.load(file)
    print(len(data))

    # count = 0
    # for obj in data:
    #     count += len(obj)
print(f"There are {count} key-value pairs in {filename}.")

1100
There are 600 key-value pairs in image_object_recognition_master.json.


Exception in thread Thread-7:
Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/site-packages/requests/models.py", line 910, in json
    return complexjson.loads(self.text, **kwargs)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/__init__.py", line 346, in loads
    return _default_decoder.decode(s)
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 337, in decode
    obj, end = self.raw_decode(s, idx=_w(s, 0).end())
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/json/decoder.py", line 355, in raw_decode
    raise JSONDecodeError("Expecting value", s, err.value) from None
json.decoder.JSONDecodeError: Expecting value: line 1 column 1 (char 0)

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/daniilabbruzzese/opt/anaconda3/lib/python3.9/threading.py", line 973, in _bootstrap_inner
    self.run()
  File "/Users/d

Saved 1200 images to file image_object_recognition_master.json.
Completed Chunk 174.
Image URL List Current Index: 17500
Saved 1300 images to file image_object_recognition_master.json.
Completed Chunk 175.
Image URL List Current Index: 17600


In [34]:
import json
import pandas as pd

# assume you have a JSON object stored in a file named data.json
with open('image_object_recognition_master.json', 'r') as f:
    data = json.load(f)

# convert JSON object to dataframe
df = pd.DataFrame(data)

# print dataframe
# print(df)

In [35]:
df

,url,object_recognition
0,http://192.168.1.13:8888/Pixstory-image-164475...,"[airship, dirigible, crane]"
1,http://192.168.1.13:8888/Pixstory-image-165960...,"[web site, website, internet site, site, menu]"
2,http://192.168.1.13:8888/Pixstory-image-166434...,"[ballplayer, baseball player, baseball]"
3,http://192.168.1.13:8888/Pixstory-image-166506...,"[pot, flowerpot, picket fence, paling]"
4,http://192.168.1.13:8888/Pixstory-image-165948...,"[switch, electric switch, electrical switch]"
...,...,...
1295,http://192.168.1.13:8888/Pixstory-image-165589...,"[bonnet, poke bonnet, abaya]"
1296,http://192.168.1.13:8888/Pixstory-image-165478...,"[web site, website, internet site, site]"
1297,http://192.168.1.13:8888/Pixstory-image-164849...,"[bobsled, bobsleigh, bob]"
1298,http://192.168.1.13:8888/Pixstory-image-164649...,"[daisy, prayer rug, prayer mat]"
